<a href="https://colab.research.google.com/github/zr-morris/CrewAI-Colab/blob/main/CrewAI_PDF_RAG_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai crewai[tools]

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import chardet

# Assuming the placeholder imports match your project's structure
from crewai import Agent, Task, Crew
from crewai_tools import DirectoryReadTool, PDFSearchTool

def on_submit(b):
    api_key = api_key_widget.value
    directory_path = directory_path_widget.value
    task_description = task_description_widget.value.strip()

    if not api_key or not directory_path or not task_description:
        with output:
            clear_output()
            print("Please fill out all fields.")
        return

    try:
        os.environ["OPENAI_API_KEY"] = api_key

        # Instantiate tools with the user-provided directory path
        docs_tool = DirectoryReadTool(directory=directory_path)
        pdf_rag_tool = PDFSearchTool()

        # Create agents with the instantiated tools
        software_agent = Agent(
            role='Software Revenue Agent',
            goal='Research accounting standards for software revenue recognition',
            backstory="""You are an expert certified professional accountant with specialized knowledge about the software industry accounting standards.""",
            allow_delegation=False,
            verbose=True,
            max_iter=10,
            tools=[docs_tool, pdf_rag_tool]
        )

        # Dynamically create a task with the user input
        dynamic_task = Task(
            description=task_description,
            expected_output='A professionally formatted analytical report that answers the question completely',
            agent=software_agent,
            output_file='compiled_report_dynamic.md'
        )

        # Assemble and execute the task with Crew
        crew = Crew(
            agents=[software_agent],
            tasks=[dynamic_task],
            verbose=True
        )
        crew.kickoff()

        # Display the content of the generated markdown file
        display_markdown(dynamic_task.output_file)

    except Exception as e:
        with output:
            clear_output()
            print(f"Error: {str(e)}")

def display_markdown(file_path):
    """Load and display the markdown file content"""
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        encoding = chardet.detect(raw_data)['encoding']
        if encoding:
            content = raw_data.decode(encoding)
            with output:
                clear_output()
                display(Markdown(content))
        else:
            with output:
                clear_output()
                print("Failed to detect file encoding.")

# Create label and input widgets
api_key_label = widgets.Label('OpenAI API Key:')
api_key_widget = widgets.Text(layout=widgets.Layout(width='90%'))

directory_path_label = widgets.Label('Directory Path:')
directory_path_widget = widgets.Text(layout=widgets.Layout(width='90%'))

task_description_label = widgets.Label('Task Description:')
task_description_widget = widgets.Textarea(layout=widgets.Layout(width='90%', height='200px'))

submit_button = widgets.Button(description="Submit")
output = widgets.Output()

submit_button.on_click(on_submit)

# Display widgets with labels
display(api_key_label, api_key_widget)
display(directory_path_label, directory_path_widget)
display(task_description_label, task_description_widget)
display(submit_button, output)
